In [10]:
import numpy as np
import os
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.misc 
import scipy.io

%matplotlib inline  
#returns current working directory of a process
cwd = os.getcwd()
print ("Current folder is %s" % (cwd) )

Current folder is /home/abbas/tensorflow/Custom_Convs


In [11]:
#Load the data and initialize the varibales

loadpath = cwd + "/data/data4vgg.npz"
l = np.load(loadpath)

# See what's in here
l.files

# load the data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]

training_tensor=np.reshape(trainimg,(ntrain,112,112,3))
testing_tensor=np.reshape(testimg,(ntest,112,112,3))

In [12]:
#Reshape all the images to (112,112,3)
for i in range(ntrain):
    currimg = trainimg[i, :]
    currimg = np.reshape(currimg, [112, 112, 3])
    training_tensor[i, :, :, :] = currimg 
print ("shape of trainimg_tensor is %s" % (training_tensor.shape,))
    
for i in range(ntest):
    currimg = testimg[i, :]
    currimg = np.reshape(currimg, [112, 112, 3])
    testing_tensor[i, :, :, :] = currimg 
print ("shape of trainimg_tensor is %s" % (testing_tensor.shape,))

shape of trainimg_tensor is (69, 112, 112, 3)
shape of trainimg_tensor is (18, 112, 112, 3)


In [13]:
#Load pre-trained VGG model
def net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat("/home/abbas/tensorflow/Custom_Convs/data/imagenet-vgg-verydeep-19.mat")
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current

    assert len(net) == len(layers)
    return net, mean_pixel

def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel

In [14]:
VGG_PATH = cwd + "/data/imagenet-vgg-verydeep-19.mat"
with tf.Graph().as_default(), tf.Session() as sess:
    image = tf.placeholder('float',shape=(None,112,112,3))
    net, mean_pixel = net(VGG_PATH, image)
    train_features=net['relu5_4'].eval(feed_dict={image:training_tensor })
    print (train_features.shape)
    test_features=net['relu5_4'].eval(feed_dict={image:testing_tensor})

(69, 7, 7, 512)


In [15]:
#Vectorize the tensors
training_vec = np.ndarray((ntrain, 7*7*512))
testing_vec = np.ndarray((ntest, 7*7*512))
# training_vec=np.reshape(train_features, (ntrain, -1))
# testing_vec=np.reshape(test_features, (ntest, -1))
# Vectorize
for i in range(ntrain):
    curr_feat = train_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    training_vec[i, :] = curr_feat_vec
    
for i in range(ntest):
    curr_feat = test_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    testing_vec[i, :] = curr_feat_vec

In [16]:
# Parameters
learning_rate   = 0.001
training_epochs = 100
batch_size      = 100
display_step    = 10

# Network Defination
n_output = nclass
weights  = {
    'wd1': tf.Variable(tf.random_normal([7*7*512, 1024], stddev=0.1)),
    'wd2': tf.Variable(tf.random_normal([1024, 1024], stddev=0.1)),
    'wd3': tf.Variable(tf.random_normal([1024, 512], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([512, n_output], stddev=0.1))
}
biases   = {
    'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'bd3': tf.Variable(tf.random_normal([512], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape  to fit fully connected layer input
    fc1 = tf.reshape(x, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
  
         # Apply Dropout
    fc1= tf.nn.dropout(fc1, dropout)
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.elu(fc2)
        # Apply Dropout
#     fc2= tf.nn.dropout(fc2, dropout)
    
    fc3 = tf.add(tf.matmul(fc2, weights['wd3']), biases['bd3'])
    fc3 = tf.nn.elu(fc3)
    fc3= tf.nn.dropout(fc3, dropout)
    # Output, class prediction
    out = tf.add(tf.matmul(fc3, weights['out']), biases['out'])
    return out

In [17]:
keep_prob = tf.placeholder(tf.float32)
# tf Graph input
x = tf.placeholder(tf.float32, [None, 7*7*512])
y = tf.placeholder(tf.float32, [None, n_output])

#Construct the model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.9,beta2=0.999,epsilon=0.1,use_locking=False).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [18]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx  = np.random.randint(ntrain, size=batch_size)
        batch_xs = training_vec[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob:0.7})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob:1.})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accuracy, feed_dict={x: testing_vec, y: testlabel, keep_prob:1.})
        print (" Test accuracy: %.3f" % (test_acc))

print ("Optimization Finished!")


Epoch: 000/100 cost: 28.187860489
 Training accuracy: 0.600
 Test accuracy: 0.667
Epoch: 010/100 cost: 24.380531311
 Training accuracy: 0.610
 Test accuracy: 0.667
Epoch: 020/100 cost: 4.028455257
 Training accuracy: 0.840
 Test accuracy: 0.833
Epoch: 030/100 cost: 0.000000004
 Training accuracy: 1.000
 Test accuracy: 0.722
Epoch: 040/100 cost: 0.000000002
 Training accuracy: 1.000
 Test accuracy: 0.722
Epoch: 050/100 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 0.944
Epoch: 060/100 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 0.889
Epoch: 070/100 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 0.889
Epoch: 080/100 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 1.000
Epoch: 090/100 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 1.000
Optimization Finished!
